In [ ]:
import pandas as pd
from dateutil.relativedelta import relativedelta
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import warnings
warnings.filterwarnings("ignore")
import math
import os
import time
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import xlsxwriter
from matplotlib.ticker import MaxNLocator
from matplotlib.backends.backend_pdf import PdfPages
start_time = time.perf_counter()

In [22]:
price_data = pd.read_csv('stockPriceData.csv')
# etf_indices = pd.read_csv('etf_indices_18_2_2025.csv')
# price_data = price_data[~(price_data['Symbol'].isin(etf_indices['Symbol']))]
price_data['Date'] = pd.to_datetime(price_data['Date'])


# Ensure the DataFrame is sorted by Symbol and Date
price_data = price_data.sort_values(by=['Date'], ascending=True)

# Calculate the 6-month rolling average of Mcap for each symbol

top_1000 =  price_data.groupby('Date', group_keys= False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
price_data_500 = price_data[price_data['Symbol'].isin(top_1000['Symbol'])]

df = price_data_500[['Date','Symbol', 'Close', 'Mcap','Volume']]

df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

df = df.sort_values(by=['Date','Mcap'], ascending=[True,False])
df = df.sort_values(['Symbol', 'Date'])
df['PrevClose'] = df.groupby('Symbol')['Close'].shift(1)
df['returns'] = (df['Close'] - df['PrevClose']) / df['PrevClose']
df = df[df.index >= '2006-01-01']


In [23]:
# Your existing code for data preparation
nifty500_df = pd.read_csv('nifty_500_index_data.csv')
nifty500_df = nifty500_df.drop(nifty500_df.columns[0], axis=1)
nifty500_df = nifty500_df.rename(columns={
    'index_date': 'Date',
    'index_value': 'Nifty 500 '
})
nifty500_df['Date'] = pd.to_datetime(nifty500_df['Date'])
nifty500_df = nifty500_df[['Date','Nifty 500 ']]
nifty500_df['Date'] = pd.to_datetime(nifty500_df['Date'], format='%d-%m-%Y')
nifty500_df['Date'] = nifty500_df['Date'].dt.strftime('%Y-%m-%d')
nifty500_df['Date'] = pd.to_datetime(nifty500_df['Date'])
nifty500_df.set_index('Date', inplace=True)

# Calculate daily returns for securities
df['Security_Daily_Return'] = df.groupby('Symbol')['Close'].pct_change()

# Calculate market daily returns
nifty500_df['Market_Daily_Return'] = nifty500_df['Nifty 500 '].pct_change()
nifty500_df = nifty500_df.reset_index()

# Merge the security dataframe with the market dataframe on Date
df = pd.merge(df, nifty500_df, on='Date')

# Define rolling windows for different time periods (in trading days)
rolling_windows = {
    '1M': 21,    # 1 month ≈ 21 trading days
    '2M': 42,    # 2 months ≈ 42 trading days
    '3M': 63,    # 3 months ≈ 63 trading days
    '6M': 126,   # 6 months ≈ 126 trading days
    '12M': 252   # 12 months ≈ 252 trading days
}

# Enhanced rolling beta calculation function
def calculate_rolling_beta(group, window):
    """
    Calculate rolling beta for a given window
    Beta = Covariance(Security Returns, Market Returns) / Variance(Market Returns)
    """
    # Calculate rolling covariance between security and market returns
    rolling_cov = group['Security_Daily_Return'].rolling(window=window).cov(
        group['Market_Daily_Return']
    )
    
    # Calculate rolling variance of market returns
    rolling_var = group['Market_Daily_Return'].rolling(window=window).var()
    
    # Calculate beta (handle division by zero)
    beta = rolling_cov / rolling_var
    
    return beta

# Calculate beta for all time periods
print("Calculating beta for multiple time periods...")

for period, window in rolling_windows.items():
    print(f"Calculating {period} beta (window: {window} days)...")
    
    # Calculate rolling beta for each security
    df[f'Beta_{period}'] = (
        df.groupby('Symbol', group_keys=False)
        .apply(lambda group: calculate_rolling_beta(group, window))
    )
    
    # Calculate rolling average returns for reference
    df[f'Avg_{period}_Security_Returns'] = (
        df.groupby('Symbol')['Security_Daily_Return']
        .apply(lambda x: x.rolling(window=window).mean())
        .reset_index(level=0, drop=True)
    )

# Calculate market rolling average returns for all periods
for period, window in rolling_windows.items():
    nifty500_df[f'Avg_{period}_Market_Returns'] = (
        nifty500_df['Market_Daily_Return'].rolling(window=window).mean()
    )

# Merge updated market data back to main dataframe
df = df.drop(columns=['Market_Daily_Return'], errors='ignore')  # Remove old column to avoid duplicates
df = pd.merge(df, nifty500_df, on='Date', how='left')

# Display beta columns
beta_columns = [col for col in df.columns if col.startswith('Beta_')]
print(f"\nBeta columns created: {beta_columns}")

# WINSORIZATION: Cap bottom and top 5% values cross-sectionally each day
print("\nApplying cross-sectional winsorization (5th and 95th percentiles)...")

def winsorize_cross_sectional(group, column, lower_pct=0.05, upper_pct=0.95):
    """
    Winsorize values cross-sectionally for a given date
    
    Parameters:
    group: DataFrame group for a specific date
    column: Column name to winsorize
    lower_pct: Lower percentile threshold (default 0.05 for 5%)
    upper_pct: Upper percentile threshold (default 0.95 for 95%)
    """
    # Get non-null values for the column
    valid_values = group[column].dropna()
    
    if len(valid_values) == 0:
        return group[column]  # Return original if no valid values
    
    # Calculate percentiles
    lower_threshold = valid_values.quantile(lower_pct)
    upper_threshold = valid_values.quantile(upper_pct)
    
    # Apply winsorization
    winsorized = group[column].copy()
    winsorized = winsorized.clip(lower=lower_threshold, upper=upper_threshold)
    
    return winsorized

# Apply winsorization to each beta column
for beta_col in beta_columns:
    print(f"Winsorizing {beta_col}...")
    
    # Create winsorized version
    winsorized_col = f'{beta_col}_Winsorized'
    
    # Group by date and apply winsorization
    df[winsorized_col] = (
        df.groupby('Date', group_keys=False)
        .apply(lambda group: winsorize_cross_sectional(group, beta_col))
        .values
    )

# Update beta columns list to include winsorized versions
winsorized_beta_columns = [col for col in df.columns if col.startswith('Beta_') and col.endswith('_Winsorized')]
all_beta_columns = beta_columns + winsorized_beta_columns

print(f"Winsorized beta columns created: {winsorized_beta_columns}")

# Compare original vs winsorized statistics
print("\nComparison: Original vs Winsorized Beta Statistics:")
print("=" * 80)

for i, original_col in enumerate(beta_columns):
    winsorized_col = f'{original_col}_Winsorized'
    
    original_data = df[original_col].dropna()
    winsorized_data = df[winsorized_col].dropna()
    
    if len(original_data) > 0 and len(winsorized_data) > 0:
        print(f"\n{original_col}:")
        print(f"{'Metric':<20} {'Original':<12} {'Winsorized':<12} {'Change':<10}")
        print("-" * 60)
        
        # Calculate metrics
        orig_mean = original_data.mean()
        wins_mean = winsorized_data.mean()
        orig_std = original_data.std()
        wins_std = winsorized_data.std()
        orig_min = original_data.min()
        wins_min = winsorized_data.min()
        orig_max = original_data.max()
        wins_max = winsorized_data.max()
        
        # Display comparison
        print(f"{'Mean':<20} {orig_mean:<12.4f} {wins_mean:<12.4f} {((wins_mean-orig_mean)/orig_mean*100):>+8.2f}%")
        print(f"{'Std Dev':<20} {orig_std:<12.4f} {wins_std:<12.4f} {((wins_std-orig_std)/orig_std*100):>+8.2f}%")
        print(f"{'Min':<20} {orig_min:<12.4f} {wins_min:<12.4f} {((wins_min-orig_min)/abs(orig_min)*100):>+8.2f}%")
        print(f"{'Max':<20} {orig_max:<12.4f} {wins_max:<12.4f} {((wins_max-orig_max)/abs(orig_max)*100):>+8.2f}%")
        
        # Count of winsorized values
        lower_wins = (df[original_col] < df[winsorized_col]).sum()
        upper_wins = (df[original_col] > df[winsorized_col]).sum()
        total_wins = lower_wins + upper_wins
        total_obs = len(winsorized_data)
        
        print(f"{'Winsorized (Lower)':<20} {lower_wins:<12,} {'':<12} {(lower_wins/total_obs*100):>8.2f}%")
        print(f"{'Winsorized (Upper)':<20} {upper_wins:<12,} {'':<12} {(upper_wins/total_obs*100):>8.2f}%")
        print(f"{'Total Winsorized':<20} {total_wins:<12,} {'':<12} {(total_wins/total_obs*100):>8.2f}%")

# Optional: Create a sample view of the results
print("\nSample of calculated betas (showing both original and winsorized):")
print("=" * 80)
sample_cols = ['Date', 'Symbol'] + all_beta_columns
if len(df) > 0:
    # Show last 5 rows with all beta calculations
    sample_df = df[sample_cols].tail(5)
    print(sample_df.to_string(index=False))

# Enhanced function to get beta for a specific stock and time period
def get_stock_beta(symbol, period='12M', latest=True, winsorized=True):
    """
    Get beta for a specific stock and time period
    
    Parameters:
    symbol: Stock symbol
    period: Time period ('1M', '2M', '3M', '6M', '12M')
    latest: If True, returns latest beta; if False, returns all betas
    winsorized: If True, returns winsorized beta; if False, returns original beta
    """
    stock_data = df[df['Symbol'] == symbol]
    
    if winsorized:
        beta_col = f'Beta_{period}_Winsorized'
    else:
        beta_col = f'Beta_{period}'
    
    if beta_col not in df.columns:
        print(f"Beta for period {period} ({'winsorized' if winsorized else 'original'}) not calculated")
        return None
    
    if latest:
        # Return the most recent non-null beta
        latest_beta = stock_data[beta_col].dropna().iloc[-1] if len(stock_data[beta_col].dropna()) > 0 else None
        return latest_beta
    else:
        # Return all beta values for the stock
        return stock_data[['Date', beta_col]].dropna()

# Function to analyze winsorization impact for a specific date
def analyze_winsorization_by_date(target_date, beta_period='12M'):
    """
    Analyze the impact of winsorization for a specific date
    
    Parameters:
    target_date: Date to analyze (string in 'YYYY-MM-DD' format)
    beta_period: Beta period to analyze
    """
    target_date = pd.to_datetime(target_date)
    date_data = df[df['Date'] == target_date]
    
    if len(date_data) == 0:
        print(f"No data found for date {target_date}")
        return
    
    original_col = f'Beta_{beta_period}'
    winsorized_col = f'Beta_{beta_period}_Winsorized'
    
    if original_col not in df.columns or winsorized_col not in df.columns:
        print(f"Beta columns for period {beta_period} not found")
        return
    
    original_values = date_data[original_col].dropna()
    winsorized_values = date_data[winsorized_col].dropna()
    
    if len(original_values) == 0:
        print(f"No valid beta values for date {target_date}")
        return
    
    # Calculate percentiles
    p5 = original_values.quantile(0.05)
    p95 = original_values.quantile(0.95)
    
    # Count winsorized values
    lower_winsorized = (original_values < p5).sum()
    upper_winsorized = (original_values > p95).sum()
    
    print(f"\nWinsorization Analysis for {target_date.strftime('%Y-%m-%d')} - {beta_period} Beta:")
    print("-" * 60)
    print(f"Total securities: {len(original_values)}")
    print(f"5th percentile threshold: {p5:.4f}")
    print(f"95th percentile threshold: {p95:.4f}")
    print(f"Values winsorized at lower bound: {lower_winsorized} ({lower_winsorized/len(original_values)*100:.1f}%)")
    print(f"Values winsorized at upper bound: {upper_winsorized} ({upper_winsorized/len(original_values)*100:.1f}%)")
    print(f"Total values winsorized: {lower_winsorized + upper_winsorized} ({(lower_winsorized + upper_winsorized)/len(original_values)*100:.1f}%)")

# Example usage of the enhanced functions
print(f"\nExample - Latest beta values for first stock in dataset:")
if len(df) > 0:
    first_symbol = df['Symbol'].iloc[0]
    print(f"Stock: {first_symbol}")
    print(f"{'Period':<8} {'Original':<12} {'Winsorized':<12} {'Difference':<12}")
    print("-" * 50)
    
    for period in rolling_windows.keys():
        orig_beta = get_stock_beta(first_symbol, period, winsorized=False)
        wins_beta = get_stock_beta(first_symbol, period, winsorized=True)
        
        if orig_beta is not None and wins_beta is not None:
            diff = wins_beta - orig_beta
            print(f"{period:<8} {orig_beta:<12.4f} {wins_beta:<12.4f} {diff:<+12.4f}")
        else:
            print(f"{period:<8} {'N/A':<12} {'N/A':<12} {'N/A':<12}")

# Example: Analyze winsorization for the latest date
if len(df) > 0:
    latest_date = df['Date'].max()
    analyze_winsorization_by_date(latest_date.strftime('%Y-%m-%d'), '12M')

print("\nBeta calculation and winsorization completed!")
print(f"Final dataset shape: {df.shape}")
print(f"Total beta columns (original + winsorized): {len(all_beta_columns)}")

Calculating beta for multiple time periods...
Calculating 1M beta (window: 21 days)...
Calculating 2M beta (window: 42 days)...
Calculating 3M beta (window: 63 days)...
Calculating 6M beta (window: 126 days)...
Calculating 12M beta (window: 252 days)...

Beta columns created: ['Beta_1M', 'Beta_2M', 'Beta_3M', 'Beta_6M', 'Beta_12M']

Applying cross-sectional winsorization (5th and 95th percentiles)...
Winsorizing Beta_1M...
Winsorizing Beta_2M...
Winsorizing Beta_3M...
Winsorizing Beta_6M...
Winsorizing Beta_12M...
Winsorized beta columns created: ['Beta_1M_Winsorized', 'Beta_2M_Winsorized', 'Beta_3M_Winsorized', 'Beta_6M_Winsorized', 'Beta_12M_Winsorized']

Comparison: Original vs Winsorized Beta Statistics:

Beta_1M:
Metric               Original     Winsorized   Change    
------------------------------------------------------------
Mean                 0.9665       0.9671          +0.07%
Std Dev              1.1917       0.7780         -34.72%
Min                  -428.6005    -2.24

In [24]:
# Function to calculate log returns
def calculate_log_returns(df):
    df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
    return df.dropna()

# Function to calculate momentum ratios
def calculate_momentum_ratios(series, period):
    return series / series.shift(period) - 1

# Define periods for momentum ratios (in trading days)
periods = {
    'MR1': 21,   # 1 month
    'MR2': 42,   # 2 months
    'MR3': 63,   # 3 months
    'MR6': 126,  # 6 months
    'MR12': 252, # 12 months
}

# Apply log return calculation
df = df.groupby('Symbol', group_keys=False).apply(calculate_log_returns)

# Calculate momentum ratios for each period
print("Calculating momentum ratios...")
for label, period in periods.items():
    df[label] = df.groupby('Symbol')['Close'].transform(lambda x: calculate_momentum_ratios(x, period))

# WINSORIZATION: Cap bottom and top 5% values cross-sectionally each day for momentum ratios
print("\nApplying cross-sectional winsorization to momentum ratios (5th and 95th percentiles)...")

def winsorize_momentum_cross_sectional(group, column, lower_pct=0.05, upper_pct=0.95):
    """
    Winsorize momentum values cross-sectionally for a given date
    
    Parameters:
    group: DataFrame group for a specific date
    column: Column name to winsorize
    lower_pct: Lower percentile threshold (default 0.05 for 5%)
    upper_pct: Upper percentile threshold (default 0.95 for 95%)
    """
    # Get non-null values for the column
    valid_values = group[column].dropna()
    
    if len(valid_values) == 0:
        return group[column]  # Return original if no valid values
    
    # Calculate percentiles
    lower_threshold = valid_values.quantile(lower_pct)
    upper_threshold = valid_values.quantile(upper_pct)
    
    # Apply winsorization
    winsorized = group[column].copy()
    winsorized = winsorized.clip(lower=lower_threshold, upper=upper_threshold)
    
    return winsorized

# Apply winsorization to each momentum ratio
momentum_columns = list(periods.keys())
for momentum_label in momentum_columns:
    print(f"Winsorizing {momentum_label}...")
    
    # Create winsorized version
    winsorized_col = f'{momentum_label}_Winsorized'
    
    # Group by date and apply winsorization
    df[winsorized_col] = (
        df.groupby('Date', group_keys=False)
        .apply(lambda group: winsorize_momentum_cross_sectional(group, momentum_label))
        .values
    )

# Update momentum columns to use winsorized versions
winsorized_momentum_columns = [f'{label}_Winsorized' for label in periods.keys()]

# Compare original vs winsorized momentum statistics
print("\nComparison: Original vs Winsorized Momentum Statistics:")
print("=" * 80)

for i, original_col in enumerate(momentum_columns):
    winsorized_col = f'{original_col}_Winsorized'
    
    if original_col in df.columns and winsorized_col in df.columns:
        original_data = df[original_col].dropna()
        winsorized_data = df[winsorized_col].dropna()
        
        if len(original_data) > 0 and len(winsorized_data) > 0:
            print(f"\n{original_col} (Period: {periods[original_col]} days):")
            print(f"{'Metric':<20} {'Original':<12} {'Winsorized':<12} {'Change':<10}")
            print("-" * 60)
            
            # Calculate metrics
            orig_mean = original_data.mean()
            wins_mean = winsorized_data.mean()
            orig_std = original_data.std()
            wins_std = winsorized_data.std()
            orig_min = original_data.min()
            wins_min = winsorized_data.min()
            orig_max = original_data.max()
            wins_max = winsorized_data.max()
            
            # Display comparison
            print(f"{'Mean':<20} {orig_mean:<12.4f} {wins_mean:<12.4f} {((wins_mean-orig_mean)/abs(orig_mean)*100 if orig_mean != 0 else 0):>+8.2f}%")
            print(f"{'Std Dev':<20} {orig_std:<12.4f} {wins_std:<12.4f} {((wins_std-orig_std)/abs(orig_std)*100 if orig_std != 0 else 0):>+8.2f}%")
            print(f"{'Min':<20} {orig_min:<12.4f} {wins_min:<12.4f} {((wins_min-orig_min)/abs(orig_min)*100 if orig_min != 0 else 0):>+8.2f}%")
            print(f"{'Max':<20} {orig_max:<12.4f} {wins_max:<12.4f} {((wins_max-orig_max)/abs(orig_max)*100 if orig_max != 0 else 0):>+8.2f}%")
            
            # Count of winsorized values
            lower_wins = (df[original_col] < df[winsorized_col]).sum()
            upper_wins = (df[original_col] > df[winsorized_col]).sum()
            total_wins = lower_wins + upper_wins
            total_obs = len(winsorized_data)
            
            print(f"{'Winsorized (Lower)':<20} {lower_wins:<12,} {'':<12} {(lower_wins/total_obs*100):>8.2f}%")
            print(f"{'Winsorized (Upper)':<20} {upper_wins:<12,} {'':<12} {(upper_wins/total_obs*100):>8.2f}%")
            print(f"{'Total Winsorized':<20} {total_wins:<12,} {'':<12} {(total_wins/total_obs*100):>8.2f}%")

# Replace original momentum columns with winsorized versions for further processing
print(f"\nReplacing original momentum ratios with winsorized versions...")
for i, momentum_label in enumerate(momentum_columns):
    winsorized_col = f'{momentum_label}_Winsorized'
    if winsorized_col in df.columns:
        # Drop original and rename winsorized to original name
        df.drop(columns=[momentum_label], inplace=True)
        df.rename(columns={winsorized_col: momentum_label}, inplace=True)
        print(f"  {momentum_label} now contains winsorized values")

print("Momentum winsorization completed!")

# Define mapping between momentum periods and corresponding beta columns
momentum_beta_mapping = {
    'MR1': 'Beta_1M_Winsorized',   # 1 month momentum -> 1 month beta
    'MR2': 'Beta_2M_Winsorized',   # 2 month momentum -> 2 month beta
    'MR3': 'Beta_3M_Winsorized',   # 3 month momentum -> 3 month beta
    'MR6': 'Beta_6M_Winsorized',   # 6 month momentum -> 6 month beta
    'MR12': 'Beta_12M_Winsorized', # 12 month momentum -> 12 month beta
}

# Check which beta columns are available and create fallback mapping
available_beta_cols = [col for col in df.columns if 'Beta_' in col]
print(f"Available beta columns: {available_beta_cols}")

# Create final mapping with fallbacks
final_beta_mapping = {}
for momentum_label, preferred_beta in momentum_beta_mapping.items():
    if preferred_beta in df.columns:
        final_beta_mapping[momentum_label] = preferred_beta
        print(f"{momentum_label} -> {preferred_beta} ✓")
    else:
        # Try fallback to non-winsorized version
        fallback_beta = preferred_beta.replace('_Winsorized', '')
        if fallback_beta in df.columns:
            final_beta_mapping[momentum_label] = fallback_beta
            print(f"{momentum_label} -> {fallback_beta} (fallback) ✓")
        else:
            print(f"Warning: No suitable beta column found for {momentum_label}")
            continue

print(f"\nFinal momentum-beta mapping:")
for mom, beta in final_beta_mapping.items():
    print(f"  {mom} normalized by {beta}")

# Normalize each momentum ratio by its corresponding beta period
print("\nNormalizing momentum ratios by corresponding beta periods...")

for momentum_label in periods.keys():
    if momentum_label not in final_beta_mapping:
        print(f"Skipping {momentum_label} - no corresponding beta column available")
        continue
    
    beta_column = final_beta_mapping[momentum_label]
    
    # Create a mask for valid beta values (not NaN, not zero, and not too close to zero)
    valid_beta_mask = (
        df[beta_column].notna() & 
        (np.abs(df[beta_column]) > 0.01)  # Avoid division by very small numbers
    )
    
    # Initialize normalized momentum column
    df[f'{momentum_label}_normalized'] = np.nan
    
    # Apply normalization only where beta is valid
    df.loc[valid_beta_mask, f'{momentum_label}_normalized'] = (
        df.loc[valid_beta_mask, momentum_label] / df.loc[valid_beta_mask, beta_column]
    )
    
    # Replace the original column with normalized version
    df[momentum_label] = df[f'{momentum_label}_normalized']
    df.drop(columns=[f'{momentum_label}_normalized'], inplace=True)
    
    print(f"  {momentum_label} normalized by {beta_column}")

# Reset index
df = df.reset_index(drop=True)

print("Calculating cross-sectional statistics...")

# Calculate the mean and std deviation of each momentum ratio across the universe
for label in periods.keys():
    df[f'mu_{label}'] = df.groupby('Date')[label].transform(lambda x: x.mean())
    df[f'sigma_{label}'] = df.groupby('Date')[label].transform(lambda x: x.std())

print("Calculating Z-scores...")

# Calculate Z-scores for each period
for label in periods.keys():
    df[f'Z_{label}'] = (df[label] - df[f'mu_{label}']) / df[f'sigma_{label}']

print("Filtering to top 500 stocks by market cap...")

# Keep only top 500 by market cap each date
df = df.groupby('Date', group_keys=False).apply(
    lambda x: x.sort_values(by='Mcap', ascending=False).head(500)
)

print("Calculating combined momentum scores...")

# Define specific combinations as tuples of the labels you want:
specific_combinations = [
    ('MR1', 'MR2', 'MR3', 'MR6', 'MR12'),  # All periods combined
    # You can add more combinations here, e.g.:
    # ('MR1', 'MR3'),      # Short-term momentum
    # ('MR6', 'MR12'),     # Long-term momentum
]

# Loop through and compute weighted avg Z-score and normalized momentum:
for comb in specific_combinations:
    # comb is now something like ('MR1', 'MR2', 'MR3', 'MR6', 'MR12')
    comb_labels = [f'Z_{label}' for label in comb]       # e.g. ['Z_MR1', 'Z_MR2', ...]
    comb_weights = np.ones(len(comb_labels)) / len(comb_labels)  # Equal weights
    comb_name = "_".join(comb)                           # e.g. 'MR1_MR2_MR3_MR6_MR12'

    # Make sure each f'Z_{label}' is actually a column in df
    missing = set(comb_labels) - set(df.columns)
    if missing:
        raise KeyError(f"The following Z-columns are missing: {missing}")

    # Weighted average Z-score
    df[f'WeightedAvgZ_{comb_name}'] = df[comb_labels].dot(comb_weights)

    # Normalized momentum score
    df[f'NormalizedMomentumScore_{comb_name}'] = np.where(
        df[f'WeightedAvgZ_{comb_name}'] >= 0,
        1 + df[f'WeightedAvgZ_{comb_name}'],
        (1 - df[f'WeightedAvgZ_{comb_name}']) ** -1
    )

print("Momentum calculation completed with period-specific beta normalization!")
print("Note: All momentum ratios were winsorized cross-sectionally (5th-95th percentiles) before beta normalization")

# Display summary statistics
print("\nSummary Statistics:")
print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"Number of unique symbols: {df['Symbol'].nunique()}")
print(f"Processing steps: Momentum calculation → Cross-sectional winsorization → Beta normalization → Z-scores")

# Show momentum columns created
momentum_cols = [col for col in df.columns if any(mr in col for mr in periods.keys())]
print(f"\nMomentum-related columns created: {len(momentum_cols)}")

# Display sample of results
print(f"\nSample data (last 5 rows):")
sample_cols = ['Date', 'Symbol', 'Mcap'] + list(periods.keys()) + [f'Z_{label}' for label in periods.keys()]
available_cols = [col for col in sample_cols if col in df.columns]
print(df[available_cols].tail().to_string(index=False))

# Show statistics for each momentum ratio
print(f"\nFinal Momentum Ratio Statistics (Winsorized + Beta-Normalized):")
print("=" * 70)
for momentum_label in periods.keys():
    if momentum_label in df.columns and momentum_label in final_beta_mapping:
        beta_used = final_beta_mapping[momentum_label]
        valid_data = df[momentum_label].dropna()
        if len(valid_data) > 0:
            print(f"\n{momentum_label} (winsorized, then normalized by {beta_used}):")
            print(f"  Count: {len(valid_data):,}")
            print(f"  Mean: {valid_data.mean():.4f}")
            print(f"  Std: {valid_data.std():.4f}")
            print(f"  Min: {valid_data.min():.4f}")
            print(f"  Max: {valid_data.max():.4f}")
            print(f"  25th percentile: {valid_data.quantile(0.25):.4f}")
            print(f"  75th percentile: {valid_data.quantile(0.75):.4f}")

# Show the normalization summary
print(f"\nFinal Processing Summary:")
print("=" * 50)
for momentum_label, beta_col in final_beta_mapping.items():
    period_days = periods[momentum_label]
    valid_count = df[momentum_label].notna().sum()
    total_count = len(df)
    print(f"{momentum_label} ({period_days} days): Winsorized → {beta_col} normalized | Valid: {valid_count:,}/{total_count:,} ({valid_count/total_count*100:.1f}%)")

# Show final combined scores
final_score_cols = [col for col in df.columns if 'NormalizedMomentumScore_' in col]
if final_score_cols:
    print(f"\nFinal Normalized Momentum Score columns:")
    for col in final_score_cols:
        print(f"  {col}")
        valid_scores = df[col].dropna()
        if len(valid_scores) > 0:
            print(f"    Range: [{valid_scores.min():.4f}, {valid_scores.max():.4f}]")
            print(f"    Mean: {valid_scores.mean():.4f}")

print("\nData is ready for further analysis!")

Calculating momentum ratios...

Applying cross-sectional winsorization to momentum ratios (5th and 95th percentiles)...
Winsorizing MR1...
Winsorizing MR2...
Winsorizing MR3...
Winsorizing MR6...
Winsorizing MR12...

Comparison: Original vs Winsorized Momentum Statistics:

MR1 (Period: 21 days):
Metric               Original     Winsorized   Change    
------------------------------------------------------------
Mean                 0.0198       0.0141         -28.87%
Std Dev              0.2708       0.1400         -48.31%
Min                  -0.9868      -0.6349        +35.67%
Max                  136.0588     1.0853         -99.20%
Winsorized (Lower)   2,127,912                    49.70%
Winsorized (Upper)   2,124,054                    49.61%
Total Winsorized     4,251,966                    99.32%

MR2 (Period: 42 days):
Metric               Original     Winsorized   Change    
------------------------------------------------------------
Mean                 0.0421       0.0303  

In [ ]:
# # Function to calculate log returns
# def calculate_log_returns(df):
#     df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
#     return df.dropna()

# # Function to calculate momentum ratios
# def calculate_momentum_ratios(series, period):
#     return series / series.shift(period) - 1

# # Define periods for momentum ratios (in trading days)
# periods = {
#     'MR1': 21,   # 1 month
#     'MR2': 42,   # 2 months
#     'MR3': 63,   # 3 months
#     'MR6': 126,  # 6 months
#     'MR12': 252, # 12 months
# }

# # Apply log return calculation
# df = df.groupby('Symbol', group_keys=False).apply(calculate_log_returns)

# # Calculate momentum ratios for each period
# print("Calculating momentum ratios...")
# for label, period in periods.items():
#     df[label] = df.groupby('Symbol')['Close'].transform(lambda x: calculate_momentum_ratios(x, period))

# # Define mapping between momentum periods and corresponding beta columns
# momentum_beta_mapping = {
#     'MR1': 'Beta_1M_Winsorized',   # 1 month momentum -> 1 month beta
#     'MR2': 'Beta_2M_Winsorized',   # 2 month momentum -> 2 month beta
#     'MR3': 'Beta_3M_Winsorized',   # 3 month momentum -> 3 month beta
#     'MR6': 'Beta_6M_Winsorized',   # 6 month momentum -> 6 month beta
#     'MR12': 'Beta_12M_Winsorized', # 12 month momentum -> 12 month beta
# }

# # Check which beta columns are available and create fallback mapping
# available_beta_cols = [col for col in df.columns if 'Beta_' in col]
# print(f"Available beta columns: {available_beta_cols}")

# # Create final mapping with fallbacks
# final_beta_mapping = {}
# for momentum_label, preferred_beta in momentum_beta_mapping.items():
#     if preferred_beta in df.columns:
#         final_beta_mapping[momentum_label] = preferred_beta
#         print(f"{momentum_label} -> {preferred_beta} ✓")
#     else:
#         # Try fallback to non-winsorized version
#         fallback_beta = preferred_beta.replace('_Winsorized', '')
#         if fallback_beta in df.columns:
#             final_beta_mapping[momentum_label] = fallback_beta
#             print(f"{momentum_label} -> {fallback_beta} (fallback) ✓")
#         else:
#             print(f"Warning: No suitable beta column found for {momentum_label}")
#             continue

# print(f"\nFinal momentum-beta mapping:")
# for mom, beta in final_beta_mapping.items():
#     print(f"  {mom} normalized by {beta}")

# # Normalize each momentum ratio by its corresponding beta period
# print("\nNormalizing momentum ratios by corresponding beta periods...")

# for momentum_label in periods.keys():
#     if momentum_label not in final_beta_mapping:
#         print(f"Skipping {momentum_label} - no corresponding beta column available")
#         continue
    
#     beta_column = final_beta_mapping[momentum_label]
    
#     # Create a mask for valid beta values (not NaN, not zero, and not too close to zero)
#     valid_beta_mask = (
#         df[beta_column].notna() & 
#         (np.abs(df[beta_column]) > 0.01)  # Avoid division by very small numbers
#     )
    
#     # Initialize normalized momentum column
#     df[f'{momentum_label}_normalized'] = np.nan
    
#     # Apply normalization only where beta is valid
#     df.loc[valid_beta_mask, f'{momentum_label}_normalized'] = (
#         df.loc[valid_beta_mask, momentum_label] / df.loc[valid_beta_mask, beta_column]
#     )
    
#     # Replace the original column with normalized version
#     df[momentum_label] = df[f'{momentum_label}_normalized']
#     df.drop(columns=[f'{momentum_label}_normalized'], inplace=True)
    
#     print(f"  {momentum_label} normalized by {beta_column}")

# # Reset index
# df = df.reset_index(drop=True)

# print("Calculating cross-sectional statistics...")

# # Calculate the mean and std deviation of each momentum ratio across the universe
# for label in periods.keys():
#     df[f'mu_{label}'] = df.groupby('Date')[label].transform(lambda x: x.mean())
#     df[f'sigma_{label}'] = df.groupby('Date')[label].transform(lambda x: x.std())

# print("Calculating Z-scores...")

# # Calculate Z-scores for each period
# for label in periods.keys():
#     df[f'Z_{label}'] = (df[label] - df[f'mu_{label}']) / df[f'sigma_{label}']

# print("Filtering to top 500 stocks by market cap...")

# # Keep only top 500 by market cap each date
# df = df.groupby('Date', group_keys=False).apply(
#     lambda x: x.sort_values(by='Mcap', ascending=False).head(500)
# )

# print("Calculating combined momentum scores...")

# # Define specific combinations as tuples of the labels you want:
# specific_combinations = [
#     ('MR1', 'MR2', 'MR3', 'MR6', 'MR12'),  # All periods combined
#     # You can add more combinations here, e.g.:
#     # ('MR1', 'MR3'),      # Short-term momentum
#     # ('MR6', 'MR12'),     # Long-term momentum
# ]

# # Loop through and compute weighted avg Z-score and normalized momentum:
# for comb in specific_combinations:
#     # comb is now something like ('MR1', 'MR2', 'MR3', 'MR6', 'MR12')
#     comb_labels = [f'Z_{label}' for label in comb]       # e.g. ['Z_MR1', 'Z_MR2', ...]
#     comb_weights = np.ones(len(comb_labels)) / len(comb_labels)  # Equal weights
#     comb_name = "_".join(comb)                           # e.g. 'MR1_MR2_MR3_MR6_MR12'

#     # Make sure each f'Z_{label}' is actually a column in df
#     missing = set(comb_labels) - set(df.columns)
#     if missing:
#         raise KeyError(f"The following Z-columns are missing: {missing}")

#     # Weighted average Z-score
#     df[f'WeightedAvgZ_{comb_name}'] = df[comb_labels].dot(comb_weights)

#     # Normalized momentum score
#     df[f'NormalizedMomentumScore_{comb_name}'] = np.where(
#         df[f'WeightedAvgZ_{comb_name}'] >= 0,
#         1 + df[f'WeightedAvgZ_{comb_name}'],
#         (1 - df[f'WeightedAvgZ_{comb_name}']) ** -1
#     )

# print("Momentum calculation completed with period-specific beta normalization!")

# # Display summary statistics
# print("\nSummary Statistics:")
# print(f"Dataset shape: {df.shape}")
# print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
# print(f"Number of unique symbols: {df['Symbol'].nunique()}")
# print(f"Normalization approach: Period-specific beta matching")

# # Show momentum columns created
# momentum_cols = [col for col in df.columns if any(mr in col for mr in periods.keys())]
# print(f"\nMomentum-related columns created: {len(momentum_cols)}")

# # Display sample of results
# print(f"\nSample data (last 5 rows):")
# sample_cols = ['Date', 'Symbol', 'Mcap'] + list(periods.keys()) + [f'Z_{label}' for label in periods.keys()]
# available_cols = [col for col in sample_cols if col in df.columns]
# print(df[available_cols].tail().to_string(index=False))

# # Show statistics for each momentum ratio
# print(f"\nMomentum Ratio Statistics (Period-Specific Beta Normalization):")
# print("=" * 70)
# for momentum_label in periods.keys():
#     if momentum_label in df.columns and momentum_label in final_beta_mapping:
#         beta_used = final_beta_mapping[momentum_label]
#         valid_data = df[momentum_label].dropna()
#         if len(valid_data) > 0:
#             print(f"\n{momentum_label} (normalized by {beta_used}):")
#             print(f"  Count: {len(valid_data):,}")
#             print(f"  Mean: {valid_data.mean():.4f}")
#             print(f"  Std: {valid_data.std():.4f}")
#             print(f"  Min: {valid_data.min():.4f}")
#             print(f"  Max: {valid_data.max():.4f}")
#             print(f"  25th percentile: {valid_data.quantile(0.25):.4f}")
#             print(f"  75th percentile: {valid_data.quantile(0.75):.4f}")

# # Show the normalization summary
# print(f"\nNormalization Summary:")
# print("=" * 50)
# for momentum_label, beta_col in final_beta_mapping.items():
#     period_days = periods[momentum_label]
#     valid_count = df[momentum_label].notna().sum()
#     total_count = len(df)
#     print(f"{momentum_label} ({period_days} days): {beta_col} | Valid: {valid_count:,}/{total_count:,} ({valid_count/total_count*100:.1f}%)")

# # Show final combined scores
# final_score_cols = [col for col in df.columns if 'NormalizedMomentumScore_' in col]
# if final_score_cols:
#     print(f"\nFinal Normalized Momentum Score columns:")
#     for col in final_score_cols:
#         print(f"  {col}")
#         valid_scores = df[col].dropna()
#         if len(valid_scores) > 0:
#             print(f"    Range: [{valid_scores.min():.4f}, {valid_scores.max():.4f}]")
#             print(f"    Mean: {valid_scores.mean():.4f}")

# print("\nData is ready for further analysis!")

Calculating momentum ratios...
Available beta columns: ['Beta_1M', 'Beta_2M', 'Beta_3M', 'Beta_6M', 'Beta_12M', 'Beta_1M_Winsorized', 'Beta_2M_Winsorized', 'Beta_3M_Winsorized', 'Beta_6M_Winsorized', 'Beta_12M_Winsorized']
MR1 -> Beta_1M_Winsorized ✓
MR2 -> Beta_2M_Winsorized ✓
MR3 -> Beta_3M_Winsorized ✓
MR6 -> Beta_6M_Winsorized ✓
MR12 -> Beta_12M_Winsorized ✓

Final momentum-beta mapping:
  MR1 normalized by Beta_1M_Winsorized
  MR2 normalized by Beta_2M_Winsorized
  MR3 normalized by Beta_3M_Winsorized
  MR6 normalized by Beta_6M_Winsorized
  MR12 normalized by Beta_12M_Winsorized

Normalizing momentum ratios by corresponding beta periods...
  MR1 normalized by Beta_1M_Winsorized
  MR2 normalized by Beta_2M_Winsorized
  MR3 normalized by Beta_3M_Winsorized
  MR6 normalized by Beta_6M_Winsorized
  MR12 normalized by Beta_12M_Winsorized
Calculating cross-sectional statistics...
Calculating Z-scores...
Filtering to top 500 stocks by market cap...
Calculating combined momentum scores..

In [26]:

# Select the required columns: Date, Symbol, and all columns containing 'NormalizedMomentumScore'
columns_to_keep = ['Date', 'Symbol'] + [col for col in df.columns if 'NormalizedMomentumScore' in col]

# Create the new DataFrame with the selected columns
final_df = df[columns_to_keep]

# Rename the 'NormalizedMomentumScore' columns to 'MoM1', 'MoM2', 'MoM3', etc.
normalized_columns = [col for col in final_df.columns if 'NormalizedMomentumScore' in col]
rename_dict = {col: f'MoMMulti' for i, col in enumerate(normalized_columns)}

# Apply the renaming
final_df.rename(columns=rename_dict, inplace=True)

In [29]:
# === Select Top 25 per Date by final weighted score ===
top_25 = (
    final_df[['Date', 'Symbol', 'MoMMulti']]
      .sort_values(['Date', 'MoMMulti'], ascending=[True, False])
      .groupby('Date')
      .head(50)
      .reset_index(drop=True)
)

# === Output Top 25 ===
print(top_25)


             Date      Symbol  MoMMulti
0      2007-01-09        HEXT  3.050060
1      2007-01-09   KIRLOSIND  2.970796
2      2007-01-09  INDIABULLS  2.911743
3      2007-01-09        HOCL  2.760109
4      2007-01-09        IIFL  2.587121
...           ...         ...       ...
226795 2025-06-06  VINATIORGA  1.449196
226796 2025-06-06        IIFL  1.422792
226797 2025-06-06        IFCI  1.416606
226798 2025-06-06       HUDCO  1.408857
226799 2025-06-06        JLHL  1.395694

[226800 rows x 3 columns]


In [32]:
master_date = price_data.drop_duplicates(subset='Date')[['Date']].reset_index(drop=True)

In [35]:
# Define the start date (it should start from 2006-06-19)
start_date = pd.to_datetime('2007-01-09')

# Filter the dates starting from 2006-06-19 (this ensures the start date is included)
filtered_dates = master_date[master_date['Date'] >= start_date]

# Ensure the start date is included
start_date_row = filtered_dates[filtered_dates['Date'] == start_date]

# Select every 50th date from the filtered dates (starting from the filtered list after the start date)
selected_dates = filtered_dates.iloc[::44]

# Combine the start date with the selected dates
selected_dates = pd.concat([start_date_row, selected_dates])

# Drop any NaN values and sort by date to ensure it's in the correct order
selected_dates = selected_dates.dropna().sort_values(by='Date')

In [37]:
# Step 1: Ensure both are datetime.date
top_25['Date'] = pd.to_datetime(top_25['Date']).dt.date
selected_dates['Date'] = pd.to_datetime(selected_dates['Date']).dt.date

# Step 2: Get the list of unique dates
date_list = selected_dates['Date'].unique()

# Step 3: Filter
filtered_df = top_25[top_25['Date'].isin(date_list)]
filtered_df

,Date,Symbol,MoMMulti
0,2007-01-09,HEXT,3.050060
1,2007-01-09,KIRLOSIND,2.970796
2,2007-01-09,INDIABULLS,2.911743
3,2007-01-09,HOCL,2.760109
4,2007-01-09,IIFL,2.587121
...,...,...,...
226395,2025-05-27,HINDALCO,1.321052
226396,2025-05-27,GLENMARK,1.320118
226397,2025-05-27,BSOFT,1.319239
226398,2025-05-27,CESC,1.309101


In [39]:
filtered_df.to_csv('BetaAdjMom.csv', index=False)